<a href="https://colab.research.google.com/github/andersonnb6/AdressFinder-Online/blob/main/AdressFinder_Online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AdressFinder-Online**  
Constitui-se de um buscador de endereços desenvolvido em linguagem R.  
Este buscador se baseia na consulta online do Código de Endereçamento Postal (CEP) ao banco de dados diponível no site https://cepaberto.com/.

### Pré-requesitos

*   Credenciamento no site https://cepaberto.com/.
*   Gerar token de acesso ao banco de dados.
*   Ter uma tabela de CEPs em formato .csv ([exemplo](https://raw.githubusercontent.com/andersonnb6/AdressFinder-Online/main/CepTable_example.csv)).



### Definindo token de acesso banco de dados
Adicione o token a variável abaixo.

In [33]:
token_key = 'a8c99e5231f62500e671cd64a8617825'

### Definindo opções

In [34]:
options(scipen=999) # desabilitando notação científica
options(warn = -1) # ignorando warnings

### Carregando pacotes

In [35]:
# Package names
packages <- c("dplyr", "cepR", "RCurl")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))

### Carregando funções

In [36]:
sleep_sec <- function(x) { # Gerar uma espera de tempo
  p1 <- proc.time()
  Sys.sleep(x)
  proc.time() - p1 
}

### Carregando banco de dados

In [ ]:
# Carregando o banco de dados
cep_table <- getURL("https://raw.githubusercontent.com/andersonnb6/AdressFinder-Online/main/CepTable_example.csv")
df <- read.csv(text = cep_table)
df

### Executando busca e gerando resultados
De acordo com a politica do site [cepaberto.com](cepaberto.com), exite um limite de busca por período de tempo. Portanto, cada busca será realizada no intervalo de 1.5s, significando que o tempo de execução desta etapa será proporcional ao número de consultas a serem realizadas. Ao término será mostrada todas as consultas realizadas. Além disso, será gerado um arquivo .csv com todos os resultados.

In [38]:
# Listas onde serão armazenados os resultados
line <- c()
identifier <- c()
state <- c()
city <- c()
district <- c()
public_area <- c()
latitude <- c()
longitude <- c()

# Loop buscador de CEP
for (i in 1:nrow(df)) {

  # consulta no site https://cepaberto.com/
  adress <- busca_cep(cep=paste(df$CEP[i]), token = token_key)
  
  # Espera de 1.5 segundos até a próxima consulta
  sleep_sec(1.5)
  
  # Armazenando resultados da consulta
  identifier <- append(df$IDENTIFICADOR[i],identifier)
  state <-  append(adress$estado,state)
  city <-  append(adress$cidade,city)
  district <- append(adress$bairro,district)
  public_area <- append(adress$logradouro, public_area)
  latitude <- append(adress$latitude, latitude)
  longitude <- append(adress$longitude, longitude)
  line <- append(i, line)
  
  # Mostrando resultado (etapa) obtida no momento
  # Descomente a linha abaixo para acompanhar o andamento quando estiver executando fora do COLAB
  ##print(paste(line[1], identifier[1], state[1], city[1], district[1], latitude[1], longitude[1], sep = "    "))
}

# Criando tabela final
final_table <- data.frame(line, identifier, state, city, district, public_area, latitude, longitude)

# Mostrando a tabela
print(final_table)

# Salvando resultados tabulados
write.csv(final_table, file = "output_file.csv",sep = ",",quote = FALSE,row.names = FALSE, col.names = FALSE)

   line identifier state   city           district
1    14     HEM014    AM Manaus       São Raimundo
2    13     HEM013    AM Manaus Colônia Terra Nova
3    12     HEM012    AM Manaus       São Raimundo
4    11     HEM011    AM Manaus       São Raimundo
5    10     HEM010    AM Manaus Colônia Terra Nova
6     9     HEM009    AM Manaus       São Raimundo
7     8     HEM008    AM Manaus Colônia Terra Nova
8     7     HEM007    AM Manaus Colônia Terra Nova
9     6     HEM006    AM Manaus       São Raimundo
10    5     HEM005    AM Manaus       São Raimundo
11    4     HEM004  <NA>   <NA>               <NA>
12    3     HEM003    AM Manaus       São Raimundo
13    2     HEM002    AM Manaus       São Raimundo
14    1     HEM001    AM Manaus       São Raimundo
                     public_area  latitude longitude
1             Avenida Rio Branco -3.103356 -60.02035
2  Rua Professora Cândida Veigas -3.011043 -60.02017
3                Rua São Vicente -3.123904 -60.04072
4              Rua 5 de